In [ ]:
# korecipegpt_kernel
# fine-tuning with pulmu1 dataset
# specifically, pulmu1 csv dataset to qa dataset/json file

In [5]:
import pandas as pd

# 엑셀 파일의 경로
file_path = '/home/jaesung/jaesung/pulmuone/llama3_ft/completion/data/pulmu1_dataset_v2.xlsx'

# 모든 시트를 로드하고 결합하는 함수
def load_and_concat_sheets(file_path):
    # 엑셀 파일의 모든 시트를 딕셔너리 형태로 로드
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    
    # 모든 시트의 데이터를 담을 리스트
    frames = []
    
    # 각 시트의 데이터를 리스트에 추가
    for sheet_name, data in all_sheets.items():
        data['SheetName'] = sheet_name  # 각 데이터프레임에 시트 이름 열 추가 (선택사항)
        frames.append(data)
    
    # 모든 데이터를 하나의 데이터프레임으로 결합
    result = pd.concat(frames, ignore_index=True)
    
    return result

# 엑셀 파일 로드 및 결합
combined_df = load_and_concat_sheets(file_path)
print(combined_df.head())  # 결합된 데이터프레임의 첫 5행 출력

          REG_NO  item_id              item_nm food_typ_cd  food_typ_nm  \
0  MI-231120-006   8272.0  당뇨케어 밀플랜_소고기볶음밥(엄지)         NaN  당뇨환자용 식단형식품   
1  MI-231120-006   8272.0  당뇨케어 밀플랜_소고기볶음밥(엄지)         NaN  당뇨환자용 식단형식품   
2  MI-231120-006   8272.0  당뇨케어 밀플랜_소고기볶음밥(엄지)         NaN  당뇨환자용 식단형식품   
3  MI-231120-006   8272.0  당뇨케어 밀플랜_소고기볶음밥(엄지)         NaN  당뇨환자용 식단형식품   
4  MI-231120-006   8272.0  당뇨케어 밀플랜_소고기볶음밥(엄지)         NaN  당뇨환자용 식단형식품   

  open_yn    reg_id emp_nm   dept_nm  dept_id  ... mat_nm         origins  \
0       N  158763.0    정민기  CTO_FMD팀  27478.0  ...   양조간장             국내산   
1       N  158763.0    정민기  CTO_FMD팀  27478.0  ...    참기름  인도,파키스탄,에티오피아등   
2       N  158763.0    정민기  CTO_FMD팀  27478.0  ...  표고버섯채             중국산   
3       N  158763.0    정민기  CTO_FMD팀  27478.0  ...     마늘             중국산   
4       N  158763.0    정민기  CTO_FMD팀  27478.0  ...     현미             국내산   

   prnt_id contents              reg_dt              mod_dt  \
0      0.0     1.25 202

In [ ]:
%%capture

# %pip install scikit-learn
# %pip install -U sentence-transformers

In [6]:
import copy

df = copy.deepcopy(combined_df)

In [7]:
def get_non_zero_columns(pivot_df):
    non_zero_columns = {}
    for index, row in pivot_df.iterrows():
        non_zero_columns[index] = row[row != 0].index.tolist()
    return non_zero_columns

In [8]:
item_mat_df = pd.pivot_table(df, values='contents', index='item_nm', columns='mat_nm', fill_value=0)

item_mat_dict= get_non_zero_columns(item_mat_df)

In [9]:
df.head(2)

,REG_NO,item_id,item_nm,food_typ_cd,food_typ_nm,open_yn,reg_id,emp_nm,dept_nm,dept_id,...,mat_nm,origins,prnt_id,contents,reg_dt,mod_dt,reg_dt_1,"fn_level(A.reg_no, B.prnt_id)",ing_typ_nm,SheetName
0,MI-231120-006,8272.0,당뇨케어 밀플랜_소고기볶음밥(엄지),NaN,당뇨환자용 식단형식품,N,158763.0,정민기,CTO_FMD팀,27478.0,...,양조간장,국내산,0.0,1.25,2023-11-20 15:02:38,2023-11-20 15:35:19,2023-11-20 15:02:04,1차,양조간장,냉동밥
1,MI-231120-006,8272.0,당뇨케어 밀플랜_소고기볶음밥(엄지),NaN,당뇨환자용 식단형식품,N,158763.0,정민기,CTO_FMD팀,27478.0,...,참기름,"인도,파키스탄,에티오피아등",0.0,1.54,2023-11-20 15:02:38,2023-11-20 15:35:19,2023-11-20 15:02:04,1차,참기름,냉동밥


In [10]:
import pandas as pd

# 예제 데이터프레임 (이미지 내용을 기반으로)
data = {
    'item_nm': list(df['item_nm']),
    'SheetName': list(df['SheetName']),
    'origins': list(df['origins']),
    'food_typ_nm': list(df['food_typ_nm']),
    'ing_typ_nm': list(df['ing_typ_nm']),
}

df = pd.DataFrame(data)

# 'item_nm'으로 그룹화
grouped = df.groupby('item_nm').agg({
    'SheetName': 'first',
    'origins': lambda x: list(x),
    'food_typ_nm': 'first',  # food_typ_nm은 동일한 값을 가질 경우 first()를 사용하여 문자열로 반환
    'ing_typ_nm': lambda x: list(x)  # ing_typ_nm은 리스트로 반환
}).reset_index()

In [11]:
grouped.head()

,item_nm,SheetName,origins,food_typ_nm,ing_typ_nm
0,'23 맑은 소고기 어묵전골,수산물,"[국내산, 국산, 국내산, 국산, 국산, 미국산, 국내산, 국산, 리투아니아, 태국...",어묵,"[식용유지, 일반식품기타가공품, 정제소금, 채소류, 농산물, 두류가공품, 농산물, ..."
1,'23 진한 가쓰오 어묵전골,수산물,"[국내산, 국산, 국내산, 국산, 국산, 미국산, 미국산, 국내산, 국산, 리투아니...",어묵,"[식용유지, 일반식품기타가공품, 정제소금, 채소류, 농산물, 연육, 두류가공품, 농..."
2,'23 풀스키즈_ 쑥 바람 바람떡,떡류,"[국산, 국내산, 국내산, 국산, 국산, 국내제조, 국산]",떡류,"[농산물, 농산물조림, 기타가공품, 재제소금, 기타가공품, 대두유, 정제수]"
3,'23 풀스키즈_사각 쑥 절편,떡류,"[국산, 국내산, 국산, 국산, 국내제조, 국산]",떡류,"[농산물, 기타가공품, 재제소금, 기타가공품, 대두유, 정제수]"
4,(GS전용)풀제주도콩 콩나물250g,농산물,"[국산, 국산, 국산, 국산]",농산물,"[농산물, 농산물, 농산물, 농산물]"


In [12]:
# 'item_nm'을 key로, 나머지 컬럼을 value로 가지는 딕셔너리 생성
side_info_dict = grouped.set_index('item_nm').T.to_dict()

In [13]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('bespin-global/klue-sroberta-base-continue-learning-by-mnr')

/home/jaesung/anaconda3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/jaesung/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
def similar_topn(item_str, ing_list, n):
    # 예제 식재료 리스트
    ingredients =  ing_list # ['ing1', 'ing2', ..]
    # 비교할 식품명
    food_name = item_str # item_nm

    # 임베딩 벡터 생성
    food_embedding = model.encode(food_name)
    ingredient_embeddings = {ingredient: model.encode(ingredient) for ingredient in ingredients}

    # 코사인 유사도 계산
    similarities = {}
    for ingredient, embedding in ingredient_embeddings.items():
        sim = cosine_similarity([food_embedding], [embedding])[0][0]
        similarities[ingredient] = sim

    # 유사도가 큰 순서대로 정렬하여 가장 유사한 식재료 2~3개 선택
    most_similar_ingredients = sorted(similarities, key=similarities.get, reverse=True)[:n]

    # print(f"'{food_name}'와(과) 가장 유사한 식재료는: {most_similar_ingredients}")

    return food_name, most_similar_ingredients

In [15]:
topn_dict = {}
for i, (k, v) in enumerate(item_mat_dict.items()):
    food_nm, topn= similar_topn(k, v, 3)
    topn_dict[food_nm] = topn

In [18]:
# sample prompt

cnt = 0
for (k1, v1), (k2, v2), (k3, v3) in zip(item_mat_dict.items(), side_info_dict.items(), topn_dict.items()):
    print(k1)
    print(v1)
    print(k2)
    print(v2)
    print(k3)
    print(v3)

    print('')
    print(f"{v3}, ['{v2['SheetName']}'], ['{v2['food_typ_nm']}']")
    cnt += 1
    if cnt > 0: break


'23 맑은 소고기 어묵전골
['고추', '냉동당근', '냉동연육', '당면', '대두단백', '대두유', '두부', '마늘', '밀가루', '베트남쌀국수액상스프', '소맥전분', '양파', '연육 명태 A', '유산균배양액혼합분말', '전분가공품', '정백당', '정제염', '폴리인산나트륨', '핵산아이지', '혼합제제']
'23 맑은 소고기 어묵전골
{'SheetName': '수산물', 'origins': ['국내산', '국산', '국내산', '국산', '국산', '미국산', '국내산', '국산', '리투아니아', '태국', '미국산', '중국산', '국내제조', '중국제조', '인도네시아', '중국산', '중국산', '태국산', '중국', '밀:미국,호주산', '국내산', '미국산'], 'food_typ_nm': '어묵', 'ing_typ_nm': ['식용유지', '일반식품기타가공품', '정제소금', '채소류', '농산물', '두류가공품', '농산물', '백설탕', '기타전분', '전분가공품', '수산물가공품', '수산물가공품', '두부', '면류', '식품첨가물', '식품첨가물(화학적합성품)', '농산물(당근)', '식품첨가물(혼합제제류)', '두류가공품', '농산물가공품', '소스류', '연육']}
'23 맑은 소고기 어묵전골
['냉동연육', '베트남쌀국수액상스프', '소맥전분']

['냉동연육', '베트남쌀국수액상스프', '소맥전분'], ['수산물'], ['어묵']


In [19]:
import json

def transform_dict_to_qa_format(item_dict, side_dict, topn_dict):
    qa_list = []
    for (k1, v1), (k2, v2), (k3, v3) in zip(item_dict.items(), side_dict.items(), topn_dict.items()):
        # 다양한 질문 형식을 생성
        questions = [
            f"{v3}, ['{v2['SheetName']}'], ['{v2['food_typ_nm']}']"
        ]
        answer = ", ".join(v1)
        # 각 질문 형식을 리스트에 추가
        for question in questions:
            qa_list.append({
                "QUESTION": question,
                "ANSWER": answer
            })
    return qa_list

# 변환된 데이터
qa_dataset = transform_dict_to_qa_format(item_mat_dict, side_info_dict, topn_dict)

# JSON 문자열로 변환
json_result = json.dumps(qa_dataset, indent=4, ensure_ascii=False)

# 결과 출력
print(json_result)
    

[
    {
        "QUESTION": "['냉동연육', '베트남쌀국수액상스프', '소맥전분'], ['수산물'], ['어묵']",
        "ANSWER": "고추, 냉동당근, 냉동연육, 당면, 대두단백, 대두유, 두부, 마늘, 밀가루, 베트남쌀국수액상스프, 소맥전분, 양파, 연육 명태 A, 유산균배양액혼합분말, 전분가공품, 정백당, 정제염, 폴리인산나트륨, 핵산아이지, 혼합제제"
    },
    {
        "QUESTION": "['가쓰오분말스프', '전분가공품', '염화칼륨'], ['수산물'], ['어묵']",
        "ANSWER": "가쓰오분말스프, 고추, 냉동당근, 냉동연육, 당면, 대두단백, 대두유, 두부, 마늘, 밀가루, 소맥전분, 양파, 연육 명태 A, 염화칼륨, 유산균배양액혼합분말, 전분가공품, 정백당, 정제염, 폴리인산나트륨, 핵산아이지, 혼합제제"
    },
    {
        "QUESTION": "['쑥가루', '냉동쑥', '한주꽃소금'], ['떡류'], ['떡류']",
        "ANSWER": "가당백앙금, 냉동쑥, 대두유, 멥쌀, 쑥가루, 정제수, 한주꽃소금"
    },
    {
        "QUESTION": "['쑥가루', '냉동쑥', '한주꽃소금'], ['떡류'], ['떡류']",
        "ANSWER": "냉동쑥, 대두유, 멥쌀, 쑥가루, 정제수, 한주꽃소금"
    },
    {
        "QUESTION": "['나물콩'], ['농산물'], ['농산물']",
        "ANSWER": "나물콩"
    },
    {
        "QUESTION": "['볶음김치향미유', '맛내기양념', '양파'], ['냉동밥'], ['즉석조리식품']",
        "ANSWER": "굴소스, 김치, 당근, 대파, 맛내기양념, 볶음김치향미유, 양배추, 양파, 옥수수유, 정백당, 정제염, 참기름, 치킨스톡P, 콩발효맛내기진, 파프리카추출색소, 핫스파이스분말"


In [20]:
qa_dataset

[{'QUESTION': "['냉동연육', '베트남쌀국수액상스프', '소맥전분'], ['수산물'], ['어묵']",
  'ANSWER': '고추, 냉동당근, 냉동연육, 당면, 대두단백, 대두유, 두부, 마늘, 밀가루, 베트남쌀국수액상스프, 소맥전분, 양파, 연육 명태 A, 유산균배양액혼합분말, 전분가공품, 정백당, 정제염, 폴리인산나트륨, 핵산아이지, 혼합제제'},
 {'QUESTION': "['가쓰오분말스프', '전분가공품', '염화칼륨'], ['수산물'], ['어묵']",
  'ANSWER': '가쓰오분말스프, 고추, 냉동당근, 냉동연육, 당면, 대두단백, 대두유, 두부, 마늘, 밀가루, 소맥전분, 양파, 연육 명태 A, 염화칼륨, 유산균배양액혼합분말, 전분가공품, 정백당, 정제염, 폴리인산나트륨, 핵산아이지, 혼합제제'},
 {'QUESTION': "['쑥가루', '냉동쑥', '한주꽃소금'], ['떡류'], ['떡류']",
  'ANSWER': '가당백앙금, 냉동쑥, 대두유, 멥쌀, 쑥가루, 정제수, 한주꽃소금'},
 {'QUESTION': "['쑥가루', '냉동쑥', '한주꽃소금'], ['떡류'], ['떡류']",
  'ANSWER': '냉동쑥, 대두유, 멥쌀, 쑥가루, 정제수, 한주꽃소금'},
 {'QUESTION': "['나물콩'], ['농산물'], ['농산물']", 'ANSWER': '나물콩'},
 {'QUESTION': "['볶음김치향미유', '맛내기양념', '양파'], ['냉동밥'], ['즉석조리식품']",
  'ANSWER': '굴소스, 김치, 당근, 대파, 맛내기양념, 볶음김치향미유, 양배추, 양파, 옥수수유, 정백당, 정제염, 참기름, 치킨스톡P, 콩발효맛내기진, 파프리카추출색소, 핫스파이스분말'},
 {'QUESTION': "['유기농맛김치', '스트링치즈p', '쌀(유기농)'], ['냉동밥'], ['즉석조리식품']",
  'ANSWER': '고추기름-1, 고춧가루, 굴소스, 당근, 대두유, 마늘, 백설탕, 스트링치즈p, 쌀(유기농), 

In [21]:
# JSONL 파일에 저장하는 함수
def save_jsonl_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        for entry in data:
            json_line = json.dumps(entry, ensure_ascii=False)
            file.write(json_line + '\n')

file_path = '/home/jaesung/jaesung/pulmuone/llama3_ft/completion/data/qa_pair_for_completion.jsonl'

save_jsonl_file(file_path, qa_dataset)

In [22]:
import json 

# JSONL 파일을 로드하는 함수
def load_jsonl_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_object = json.loads(line.strip())
            data.append(json_object)
    return data

# JSONL 파일 경로
file_path = '/home/jaesung/jaesung/pulmuone/llama3_ft/completion/data/qa_pair_for_completion.jsonl'

# JSONL 파일 로드
loaded_data = load_jsonl_file(file_path)

# 결과 출력
print(loaded_data[0])

{'QUESTION': "['냉동연육', '베트남쌀국수액상스프', '소맥전분'], ['수산물'], ['어묵']", 'ANSWER': '고추, 냉동당근, 냉동연육, 당면, 대두단백, 대두유, 두부, 마늘, 밀가루, 베트남쌀국수액상스프, 소맥전분, 양파, 연육 명태 A, 유산균배양액혼합분말, 전분가공품, 정백당, 정제염, 폴리인산나트륨, 핵산아이지, 혼합제제'}


In [23]:
print(loaded_data[1])

{'QUESTION': "['가쓰오분말스프', '전분가공품', '염화칼륨'], ['수산물'], ['어묵']", 'ANSWER': '가쓰오분말스프, 고추, 냉동당근, 냉동연육, 당면, 대두단백, 대두유, 두부, 마늘, 밀가루, 소맥전분, 양파, 연육 명태 A, 염화칼륨, 유산균배양액혼합분말, 전분가공품, 정백당, 정제염, 폴리인산나트륨, 핵산아이지, 혼합제제'}
